In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [ ]:
#export
from exp.nb_08 import *

## ConvNet

In the part one of the course, we had close to 99% accuracy on MNIST in 3 epochs with a simple CNN (lesson7-resnet notebook). Let's check we can match this.

In [ ]:
x_train,y_train,x_valid,y_valid = get_data()

x_train,x_valid = normalize_to(x_train,x_valid)
train_ds,valid_ds = Dataset(x_train, y_train),Dataset(x_valid, y_valid)

nh,bs = 50,512
c = y_train.max().item()+1
loss_func = F.cross_entropy

data = DataBunch(*get_dls(train_ds, valid_ds, bs), 1, c)

In [ ]:
sched = combine_scheds([0.25, 0.75], [sched_cos(0.1/25, 0.1), sched_cos(0.1, 0.)]) 

mnist_view = view_tfm(1,28,28)
cbfs = [Recorder,
        partial(AvgStatsCallback,accuracy),
        CudaCallback,
        partial(ParamScheduler, 'lr', sched),
        partial(BatchTransformXCallback, mnist_view)]

In [ ]:
nfs = [8,16,16,16,16]

In [ ]:
def conv_layer(ni, nf, ks=3, stride=2, **kwargs):
    return nn.Sequential(
        nn.Conv2d(ni, nf, ks, padding=ks//2, stride=stride, bias=True),
        nn.BatchNorm2d(nf),
        GeneralRelu(**kwargs))

In [ ]:
opt_func = optim.Adam

In [ ]:
learn,run = get_learn_run(nfs, data, 0.03, conv_layer,
                          opt_func=opt_func, cbs=cbfs)

In [ ]:
%time run.fit(3, learn)

train: [0.6206956640625, tensor(0.8288, device='cuda:0')]
valid: [1.06133662109375, tensor(0.7672, device='cuda:0')]
train: [0.0803643701171875, tensor(0.9761, device='cuda:0')]
valid: [0.07125917358398437, tensor(0.9790, device='cuda:0')]
train: [0.04357732421875, tensor(0.9875, device='cuda:0')]
valid: [0.04074094848632812, tensor(0.9886, device='cuda:0')]
CPU times: user 8 s, sys: 1.82 s, total: 9.82 s
Wall time: 9.75 s
